# 2019 OK JPL Horizon vs OpenOrb comparison

## Assumptions/ accountability-sourced documentation:

* Values for chosen orbit found here: https://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2019%20OK
* Location: I11, Gemini South
* Time: 2010/01/01 to 2020/01/01, midnights

## Current questions/ issues:

* Do I need to cite things in-code? nah, but track examples/ code snippets
* Do we need a maximum difference function or class? function
* check data to stop kernel from dying

# Display-ready Zone

## Set-up

In [1]:
%matplotlib inline

import sys
import re
import numpy as np
import pandas as pd
import astropy as ap
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import QTable
import matplotlib.pyplot as plt
import astroquery as aq
from astroquery.jplhorizons import Horizons
import pyoorb as oo
oo.pyoorb.oorb_init()

0

In [10]:
# "Global" variables: times (UTC), location, object (find a way to generalize objects)
start_time = '2010-01-01T00:00:00'
stop_time = '2020-01-01T00:00:01'
internal_start = ap.time.Time(start_time)
internal_stop = ap.time.Time(stop_time)
element_time = internal_start.jd
Gemini_S = 'I11@399'
obj_id = 'Ivezic'

## JPL Horizons

In [11]:
def get_ephem_jpl():
    el_obj = Horizons(id=obj_id, location= '500@10',
               epochs=element_time)
    el_jpl = el_obj.elements()
    ephem_obj = Horizons(id= obj_id, location= Gemini_S,
               epochs={'start': start_time, 'stop':stop_time,
                      'step':'1d'})
    ephem_jpl = ephem_obj.ephemerides()
    
    return el_jpl, ephem_jpl

## OpenOrb/ pyoorb

In [5]:
# function to reorganize JPL Horizons output into pyoorb-acceptable inputs. Expand for multiple orbits next
def reorganizer(orbits, epoch):
    '''
    Parameters
    ------
    orbits : `~numpy.ndarray` (N, 18)
    epoch : `~numpy.ndarray` (3652, 2)
        Constrained to cometary format.
    Returns
    -------
    new_array : `~numpy.ndarray` (N, 12)
        Orbits formatted in the format expected by PYOORB. 
            orbit_id : index of input orbits
            elements x6: orbital elements of propagated orbits
            orbit_type : orbit type
            epoch_mjd : epoch of the propagate orbit
            time_scale : time scale of output epochs
            H/M1 : absolute magnitude
            G/K1 : photometric slope parameter
    '''
    
    temp = orbits.copy()
    temp = temp.as_array().data
    if temp.shape == (6,):
        num_orbits = 1
    else:
        num_orbits = temp.shape[0]
        
    for i in range(num_orbits):
        ids = i
        orbit_type = 2
        time_scale = 3
        
    # elements x6
    q = temp[0][6]
    e = temp[0][5]
    incl = np.deg2rad(temp[0][7])
    longnode = np.deg2rad(temp[0][8])
    argper = np.deg2rad(temp[0][9])
    peri_epoch = ap.time.Time(temp[0][10], format='jd').mjd

    mag = temp[0][3]
    slope = temp[0][4]
    
    if num_orbits > 1:
        new_array = np.array(
            np.array([
                ids, 
                q,
                e,
                incl,
                longnode,
                argper,
                peri_epoch,
                orbit_type,
                epoch,
                time_scale,
                mag,
                slope
            ]), 
            dtype=np.double, 
            order='F')
    else:
        new_array = np.array(
            [[
                ids, 
                q,
                e,
                incl,
                longnode,
                argper,
                peri_epoch,
                orbit_type,
                epoch,
                time_scale,
                mag,
                slope
            ]], 
            dtype=np.double,
            order='F')
    
    return new_array

In [6]:
def get_ephem_OpenOrb():
    # time conversions, epochs for pyoorb to work
    element_time_pyoorb = internal_start.mjd
    start_pyoorb = internal_start.mjd
    stop_pyoorb = internal_stop.mjd
    peri_time = ap.time.Time(el_jpl['Tp_jd'][0], format='jd').mjd
    #print(start_pyoorb, stop_pyoorb)
    
    pyoorb_formatted = reorganizer(el_jpl, start_pyoorb)
    t0 = np.array([element_time_pyoorb, 1], dtype=np.double, order='F')
    mjds = np.arange(start_pyoorb, stop_pyoorb, 1)
    epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
    ephem_pyoorb, err = oo.pyoorb.oorb_ephemeris_basic(in_orbits=pyoorb_formatted,
                                         in_obscode='I11',
                                         in_date_ephems=epochs,
                                         in_dynmodel='N')

    
    
    return ephem_pyoorb

## OrbFit

In [19]:
#%cd 2019OK
#! ./fitobs.x < ast.inp > /dev/null


import re
file = "202930.eph"
newFile = "subbed202930.eph"
pattern = re.compile(r"(-|\+) ([0-9])")
originalData = open(file).read()
subbedFile = pattern.sub(r"\1\2", originalData)
with open(newFile, "w") as newEPH:
    newEPH.write(subbedFile)
a = np.genfromtxt(newFile, skip_header=4)
RA = a[:,(5,6,7)]
print(RA) # use SkyCoord to convert it all to degrees
DEC = a[:,[8,9,10]]
print(DEC) # same as RA

[[13.     4.     6.627]
 [13.     4.    50.858]
 [13.     5.    34.292]
 ...
 [16.    56.    17.335]
 [16.    58.     7.199]
 [16.    59.    57.078]]
[[ -7.    56.    10.62]
 [ -8.     1.    53.92]
 [ -8.     7.    32.63]
 ...
 [-26.     4.    15.93]
 [-26.     7.    19.35]
 [-26.    10.    18.44]]
    RA   
   deg   
---------
196.02761
196.21191
196.39288
196.57047
196.74461
196.91523
197.08228
197.24569
197.40538
197.56129
      ...
250.87194
251.32848
251.78529
252.24234
252.69959
253.15701
253.61457
254.07225
254.53002
254.98785
Length = 3653 rows


In [20]:
def get_ephem_OrbFit():
    %cd 2019OK # this is the directory with all the files, maybe reorganize this?
    ! ./fitobs.x < ast.inp > /dev/null
    file = "202930.eph" #find a way to take the output of previous command and feed it in here
    newFile = "subbed202930.eph"
    pattern = re.compile(r"(-|\+) ([0-9])")
    originalData = open(file).read()
    subbedFile = pattern.sub(r"\1\2", originalData)
    with open(newFile, "w") as newEPH:
        newEPH.write(subbedFile)
    ephem_OrbFit = np.genfromtxt(newFile, skip_header=4)
    RA = ephem_OrbFit[:,(5,6,7)]
    print(RA) # use SkyCoord to convert it all to degrees
    DEC = ephem_OrbFit[:,[8,9,10]]
    print(DEC) # same business as RA
    return ephem_OrbFit

## Outputs and comparison

In [21]:
# eventually all will be under monster get_ephem() function
el_jpl, ephem_jpl = get_ephem_jpl()
ephem_OpenOrb = get_ephem_OpenOrb()
ephem_OrbFit = get_ephem_OrbFit()

[Errno 2] No such file or directory: '2019OK # this is the directory with all the files, maybe reorganize this?'
/astro/users/maria8ch/great-integrator-bake-off/2019OK
[[13.     4.     6.627]
 [13.     4.    50.858]
 [13.     5.    34.292]
 ...
 [16.    56.    17.335]
 [16.    58.     7.199]
 [16.    59.    57.078]]
[[ -7.    56.    10.62]
 [ -8.     1.    53.92]
 [ -8.     7.    32.63]
 ...
 [-26.     4.    15.93]
 [-26.     7.    19.35]
 [-26.    10.    18.44]]


## Difference metric

In [27]:
# difference function: how to write it so that any integrator we use can be parsed?


def max_diff(ephem1, ephem2, integrator):
    '''
    Parameters
    ----------
    ephem1: elements of ephemerides, 1st integrator, numpy array?
    ephem2: elements of ephemerides, 2nd integrator, numpy array? (same type as ephem1)
    integrator: indicates which integrators are being comparted, str (not implemented yet)
    
    Returns
    -------
    diff_matrix: separation, offset, magnitude diff between ephem1 and ephem2, numpy array
    '''
    
    #coordinates
    RA1 = ephem1['RA']
    Dec1 = ephem1['DEC']
    RA2 = ephem2[0][:,1] * u.deg
    Dec2 = ephem2[0][:,2] * u.deg
    
    coord1 = SkyCoord(RA1, Dec1, frame='icrs')
    coord2 = SkyCoord(RA2, Dec2, frame='icrs')
    sep = coord1.separation(coord2)
    
    # magnitudes
    #mag1 = ephem1['H']
    #mag2 = ephem2['H']
    
    #magnitude = mag2 - mag1
    
    # matrix
    # degs, mins, secs; extract secs
    diff_matrix = sep.arcsec
        
    return diff_matrix

In [29]:
great_circle_diff = max_diff(ephem_jpl, ephem_pyoorb,*_)
print("These are the differences:", great_circle_diff)
precision = (great_circle_diff/3600)/360
print("These are the precision values:", precision)
#HALF A PIXEL (1p is 0.2 arcs (but also photometry can improve measurements))

These are the differences: [0.00785716 0.01113825 0.00619841 ... 0.1361494  0.12737095 0.1156872 ]
These are the precision values: [6.06262453e-09 8.59433169e-09 4.78272197e-09 ... 1.05053549e-07
 9.82800574e-08 8.92648144e-08]


# Prep Zone

In [22]:
# monster function. Will take in object id's and convert&conquer to all integrator formats
def get_ephems():
    get_ephem_jpl()
    get_ephem_OpenOrb()
    get_ephem_OrbFit()
    return

In [8]:
# 2019 OK orbit and time set-up
orbit_2019OK = np.array([[0, el_jpl['q'][0], el_jpl['e'][0], np.deg2rad(el_jpl['incl'][0]),
                   np.deg2rad(el_jpl['Omega'][0]), np.deg2rad(el_jpl['w'][0]),
                   peri_time, 2, start_pyoorb, 1, el_jpl['H'][0], el_jpl['G'][0]]],
                  dtype=np.double, order='F')

t0 = np.array([element_time_pyoorb, 1], dtype=np.double, order='F')
mjds = np.arange(start_pyoorb, stop_pyoorb, 1)
epochs = np.array(list(zip(mjds, [1]*len(mjds))), dtype=np.double, order='F')
ephem_pyoorb, err = oo.pyoorb.oorb_ephemeris_basic(in_orbits=orbit_2019OK,
                                         in_obscode='I11',
                                         in_date_ephems=epochs,
                                         in_dynmodel='N')

print(ephem_pyoorb, err)

# Recipe

In [ ]:
'''
Gfortran version 4.8.6

Useful bits from docs:

JPL: 'targetname','datetime_jd','datetime_str','H','G','e','q','incl','Omega','w','Tp_jd','n','M','nu','a','Q','P'

pyoorb:
orbit id: an integer number to identify the orbit; usually ranges from 0 to n-1, where n is the number of orbits
perihelion distance (au) for COM, semimajor axis a (au) for KEP, x (au) for CART
eccentricity for COM or KEP, y (au) for CART
inclination (deg) for COM or KEP, z (au) for CART
longitude of the ascending node (deg) for COM and KEP, dx/dt (au/day) for CART
argument of the periapsis (deg) for COM and KEP, dy/dt (au/day) for CARqT
epoch of perihelion (modified Julian date) for COM, mean anomaly (deg) for KEP, dz/dt for CART
orbital element type; integer value: CART: 1, COM: 2, KEP: 3
epoch of the osculating elements (modified Julian date)
timescale type of the epochs provided; integer value: UTC: 1, UT1: 2, TT: 3, TAI: 4
absolute magnitude of the target
photometric slope parameter of the target


JPL TableColumns values to extract:
[0,1,2,3,7,8,23] = [targetname, datetime str, datettime jd, H, RA, DEC, V]

Want pyoorb.oorb_ephemeris_basic , use these indices for properties:
[0,1,2,9] = [mjd, RA (deg), DEC(deg), predicted apparent V-band mag]

example copied from--> https://github.com/oorb/oorb/tree/master/python#ephemeris-computation
'''